### Creating Dimension Tables


1. Dim Patients
2. Dim Provider
3. Dim Dates


In [18]:
create schema if not exists gold;

StatementMeta(, d9431510-bae8-4c62-aa2f-93ec2da49ada, 22, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
DROP TABLE IF EXISTS gold.dim_patients;

CREATE TABLE gold.dim_patients
USING DELTA
AS
SELECT
    ROW_NUMBER() OVER (ORDER BY patient_id) AS patient_key,
    patient_id,
    Gender,
    FLOOR(DATEDIFF(current_date(), dob) / 365) AS age,
    CASE
        WHEN FLOOR(DATEDIFF(current_date(), dob) / 365) < 18 THEN '0-17'
        WHEN FLOOR(DATEDIFF(current_date(), dob) / 365) BETWEEN 18 AND 35 THEN '18-35'
        WHEN FLOOR(DATEDIFF(current_date(), dob) / 365) BETWEEN 36 AND 55 THEN '36-55'
        ELSE '56+'
    END AS age_group
FROM dbo.patients;




StatementMeta(, d9431510-bae8-4c62-aa2f-93ec2da49ada, 18, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [16]:
SHOW TABLES IN dbo;


StatementMeta(, d9431510-bae8-4c62-aa2f-93ec2da49ada, 20, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 3 fields>

In [17]:


SELECT * FROM Healthcare_Claims_LH.gold.dim_patients LIMIT 1000

StatementMeta(, d9431510-bae8-4c62-aa2f-93ec2da49ada, 21, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 5 fields>

In [19]:
create or replace table gold.dim_Providers as
select 
row_number() over(order by provider_id) as provider_key,
provider_id, provider_name,specialty,city,state
from dbo.Providers

StatementMeta(, d9431510-bae8-4c62-aa2f-93ec2da49ada, 23, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [20]:


SELECT * FROM Healthcare_Claims_LH.gold.dim_providers LIMIT 1000

StatementMeta(, d9431510-bae8-4c62-aa2f-93ec2da49ada, 24, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 6 fields>

In [3]:
create or replace table gold.dim_Dates as
select distinct
    cast(date_format(d,'yyyyMMdd') as int) as date_key,
    d as full_date,
    Year(d) as year,
    Month(d) as month,
    Day(d)as day,
    concat('Q',Quarter(d)) as quarter,
    date_format(d,'EEEE') AS weekday
    from
    (select service_date as d from dbo.Claims
    union
    select claim_date as d from dbo.claims
    union
    select payment_date as d from dbo.Payments
    );




StatementMeta(, a28b5795-9111-4cca-9c52-589d75c10428, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
SELECT * FROM Healthcare_Claims_LH.gold.dim_dates LIMIT 1000

StatementMeta(, a28b5795-9111-4cca-9c52-589d75c10428, 5, Finished, Available, Finished)

<Spark SQL result set with 19 rows and 7 fields>

### Creating Fact Tables

In [2]:
create or replace table gold.fact_claim as
select 
c.claim_id,
p.patient_key,
pr.provider_key,
d.date_key as service_date_key,
c.claim_amount,
DATEDIFF(day, c.service_date, c.claim_date) AS days_to_submit,
case
when c.claim_status = 'Denied' then 1
else 0
end as Is_Denied
from dbo.claims c
join gold.dim_patients p on c.patient_id = p.patient_id
join gold.dim_providers pr on c.provider_id = pr.provider_id
join gold.dim_dates d on c.service_date = d.full_date;


StatementMeta(, e70d1d71-17c3-4d77-a0d9-3ce851b9f1f7, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
SELECT * FROM Healthcare_Claims_LH.gold.fact_claim LIMIT 1000

StatementMeta(, e70d1d71-17c3-4d77-a0d9-3ce851b9f1f7, 4, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 7 fields>

In [2]:
create or replace table gold.fact_payment as
select 
pmt.payment_id,
fc.claim_id,
fc.patient_key,
fc.provider_key,
d.date_key as paymentdatekey,
pmt.paid_amount
from dbo.Payments pmt 
join gold.fact_claim fc on pmt.claim_id = fc.claim_id
join gold.dim_dates d on pmt.payment_date = d.full_date;


StatementMeta(, ae54ee36-8aad-4ccc-9b31-bb0eec67b51d, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
SELECT * FROM Healthcare_Claims_LH.gold.fact_payment LIMIT 1000

StatementMeta(, ae54ee36-8aad-4ccc-9b31-bb0eec67b51d, 4, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 6 fields>

In [5]:
Create or replace table gold.fact_denials as
select
cd.denial_id,
fc.claim_id,
cd.denial_reason,
fc.patient_key,
fc.provider_key,
d.date_key as denial_date_key
from dbo.Claim_denials cd join
gold.fact_claim fc on cd.claim_id = fc.claim_id
join gold.dim_dates d on cd.denial_date = d.full_date;


StatementMeta(, ae54ee36-8aad-4ccc-9b31-bb0eec67b51d, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>